In [23]:
import numpy as np
import pandas as pd

In [2]:
# import data
df = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/subanalysis.p')
# NOTE THAT 'IN' MEAN IN TO THE ALT DISTRIBUTION, WHCIH IS NOT WHAT MATHEMATICA DOES

In [3]:
def getcount(decision, direction):
    directiondir = {'in':'-1', 'out':'1', 'unk':'0', 'fail':'2'}
    query = '%s == %s' %(decision, directiondir[direction])
    num = len(np.asarray(df.query(query)[decision], dtype=int))
    return num

In [4]:
# plot some stuff
expin = getcount('dexp', 'in')
expout = getcount('dexp', 'out')
expunk = getcount('dexp', 'unk')
lnin = getcount('dln', 'in')
lnout = getcount('dln', 'out')
lnunk = getcount('dln', 'unk')
strexpin = getcount('dstrexp', 'in')
strexpout = getcount('dstrexp', 'out')
strexpunk = getcount('dstrexp', 'unk')
plwcin = getcount('dplwc', 'in')
plwcout = getcount('dplwc', 'out')
plwcunk = getcount('dplwc', 'unk')
# order = "LN", "Exp", "Str Exp", "PLwC"
incounts = [lnin, expin, strexpin, plwcin]
outcounts = [lnout, expout, strexpout, plwcout]
unkcounts = [lnunk, expunk, strexpunk, plwcunk]
print incounts
print outcounts
print unkcounts

[415, 317, 388, 489]
[136, 1069, 852, 0]
[1239, 301, 500, 1333]


In [52]:
doms = np.unique(df.Domain)
# number of graphs and mean alpha value in each domain
counts = np.zeros_like(doms)
means = np.zeros_like(doms)
counts_plaus = np.zeros_like(doms)
means_plaus = np.zeros_like(doms)
counts_plaustail = np.zeros_like(doms)
means_plaustail = np.zeros_like(doms)
for i,dom in enumerate(doms):
    query = "Domain == '%s'" %dom
    subdf = df.query(query)
    counts[i] = len(subdf)
    means[i] = format(np.mean(subdf.alpha), '.2f')
    subdf_plaus = subdf.query("ppl>0.1")
    counts_plaus[i] = len(subdf_plaus)
    means_plaus[i] = format(np.mean(subdf_plaus.alpha), '.2f')
    subdf_plaustail = subdf_plaus.query("ntail>=50")
    counts_plaustail[i] = len(subdf_plaustail)
    means_plaustail[i] = format(np.mean(subdf_plaustail.alpha), '.2f')


In [72]:
print doms
print counts_plaustail
print means_plaustail

['Biological' 'Economic' 'Informational' 'Social' 'Technological'
 'Transportation']
[53 0 6 106 799 1]
['2.69' 'nan' '2.38' '4.74' '2.23' '4.82']


In [70]:
# import dataframe
df = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/LRTAnalysis/analysis/gmlcatalog.p')
len(df.query('Domain == "Informational"'))






23